In [ ]:
import requests
import pandas as pd
import time
import random
URL_BASE = "https://steamspy.com/api.php"
OUTPUT_FILE = "steamspy_f2p_raw.csv"
# Pausas aleatorias para no saturar el servidor
PAUSE_MIN = 0.5
PAUSE_MAX = 1.2

def steamspy_request(params):
    """Hace una request segura a SteamSpy y devuelve un dict."""
    try:
        time.sleep(random.uniform(PAUSE_MIN, PAUSE_MAX))
        response = requests.get(URL_BASE, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error en la request {params}: {e}")
        return {}

# OBTENER TOP F2P POR DESCARGAS HISTÓRICAS ---
print("Descargando Top F2P por descargas totales...")
data_f2p_downloads = steamspy_request({"request": "tag", "tag": "Free to Play"})

df_f2p_downloads = pd.DataFrame.from_dict(data_f2p_downloads, orient="index")
print(f"Descargados {len(df_f2p_downloads)} juegos F2P del tag principal.")

# OBTENER TOP GLOBAL POR HORAS JUGADAS RECIENTES ---
print("Descargando Top global por horas jugadas (2 semanas)...")
data_top_recent = steamspy_request({"request": "top100in2weeks"})

df_top_recent = pd.DataFrame.from_dict(data_top_recent, orient="index")
print(f"Descargados {len(df_top_recent)} juegos globales recientes.")

# FILTRAR F2P DE LOS TOP RECIENTES ---
df_top_recent_f2p = df_top_recent[df_top_recent["price"] == 0]
print(f"{len(df_top_recent_f2p)} juegos F2P entre los más jugados recientemente.")

# COMBINAR Y ELIMINAR DUPLICADOS ---
df_combined = pd.concat([df_f2p_downloads, df_top_recent_f2p], ignore_index=True)
df_combined.drop_duplicates(subset="appid", inplace=True)
df_combined.reset_index(drop=True, inplace=True)

print(f"Total combinado: {len(df_combined)} juegos únicos Free-to-Play.")

#  GUARDAR EN CSV CRUDO ---
df_combined.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
print(f"\n✅ Dataset guardado en: {OUTPUT_FILE}")

# --- OPCIONAL: vista previa de columnas disponibles ---
print("\nColumnas disponibles en el dataset:")
print(df_combined.columns.tolist())




Descargando Top F2P por descargas totales...
Descargados 6089 juegos F2P del tag principal.
Descargando Top global por horas jugadas (2 semanas)...
Descargados 100 juegos globales recientes.
0 juegos F2P entre los más jugados recientemente.
Total combinado: 6089 juegos únicos Free-to-Play.

✅ Dataset guardado en: steamspy_f2p_raw.csv

Columnas disponibles en el dataset:
['appid', 'name', 'developer', 'publisher', 'score_rank', 'positive', 'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount', 'ccu']


In [2]:
import pandas as pd
df = pd.read_csv("steamspy_f2p_raw.csv")
df_steam_spy = df
df_steam_spy

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,1172470,Apex Legends,Respawn,Electronic Arts,NaN,668053,326926,0,"100,000,000 .. 200,000,000",11025,635,977,256,0,0,0,124262
1,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",NaN,1520457,1037487,0,"100,000,000 .. 200,000,000",23333,820,5899,335,0,0,0,314682
2,440,Team Fortress 2,Valve,Valve,NaN,1044264,117208,0,"50,000,000 .. 100,000,000",21058,772,4175,161,0,0,0,43819
3,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,NaN,506516,48852,0,"50,000,000 .. 100,000,000",9607,1688,342,583,0,0,0,10408
4,1599340,Lost Ark,Smilegate RPG,Amazon Games,NaN,143481,58540,0,"50,000,000 .. 100,000,000",5408,1261,752,457,0,0,0,17355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084,892310,Infinity Heroes,Lightmare Studios,Lightmare Studios,NaN,32,14,0,"0 .. 20,000",2041,0,2041,0,0,0,0,0
6085,661460,Thin Judgment,James Roll,James Roll,NaN,17,15,0,"0 .. 20,000",0,0,0,0,0,0,0,0
6086,1509950,Empire IDLE,Kadragon,Kadragon,NaN,9,17,0,"0 .. 20,000",0,0,0,0,199,199,0,0
6087,1262030,猎魔战纪,"猎魔战纪, 魔域永恒",魔域永恒,NaN,3,4,0,"0 .. 20,000",0,0,0,0,0,0,0,0


In [3]:


# TRANSFORMAR owners a integers
def parse_owners(owners_str):
    """
    Convierte '0 .. 20,000' en owners_min y owners_max
    """
    if pd.isna(owners_str) or owners_str == '':
        return 0, 0
    
    # Limpiar y separar
    parts = str(owners_str).replace(',', '').split('..')
    
    try:
        owners_min = int(parts[0].strip())
        owners_max = int(parts[1].strip()) if len(parts) > 1 else owners_min
    except:
        owners_min = 0
        owners_max = 0
    
    return owners_min, owners_max

# Aplicar transformación
df_steam_spy[['owners_min', 'owners_max']] = df_steam_spy['owners'].apply(
    lambda x: pd.Series(parse_owners(x))
)

# Calcular mediana
df_steam_spy['owners_median'] = (df_steam_spy['owners_min'] + df_steam_spy['owners_max']) / 2

# Convertir otros campos a numeric si no lo son
numeric_cols = ['ccu', 'average_2weeks', 'positive', 'negative', 'average_forever', 'median_forever', 'price']
for col in numeric_cols:
    df_steam_spy[col] = pd.to_numeric(df_steam_spy[col], errors='coerce').fillna(0)

# Guardar con transformaciones
df_steam_spy.to_csv("steamspy_transformed.csv", index=False) 

In [4]:

df_clean = df_steam_spy[
    (df_steam_spy['owners_min'] > 500) &
    (df_steam_spy['average_forever'] > 0) & 
    (
        (df_steam_spy['ccu'] > 10) |
        (df_steam_spy['average_2weeks'] > 1) |
        (df_steam_spy['positive'] + df_steam_spy['negative'] > 50)
    )
].copy()
df_clean
# hasta ahora hemos quitado juegos inactivos



,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,owners_min,owners_max,owners_median
0,1172470,Apex Legends,Respawn,Electronic Arts,NaN,668053,326926,0,"100,000,000 .. 200,000,000",11025,635,977,256,0,0,0,124262,100000000,200000000,150000000.0
1,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",NaN,1520457,1037487,0,"100,000,000 .. 200,000,000",23333,820,5899,335,0,0,0,314682,100000000,200000000,150000000.0
2,440,Team Fortress 2,Valve,Valve,NaN,1044264,117208,0,"50,000,000 .. 100,000,000",21058,772,4175,161,0,0,0,43819,50000000,100000000,75000000.0
3,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,NaN,506516,48852,0,"50,000,000 .. 100,000,000",9607,1688,342,583,0,0,0,10408,50000000,100000000,75000000.0
4,1599340,Lost Ark,Smilegate RPG,Amazon Games,NaN,143481,58540,0,"50,000,000 .. 100,000,000",5408,1261,752,457,0,0,0,17355,50000000,100000000,75000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3937,796380,Terra Feminarum,Polar Night Games,Polar Night Games,NaN,54,8,0,"20,000 .. 50,000",278,0,278,0,0,0,0,0,20000,50000,35000.0
3938,651280,TinkerQuarry,Hallowraith,Caprizant Games,NaN,81,8,0,"20,000 .. 50,000",120,0,120,0,499,499,0,0,20000,50000,35000.0
3953,658510,Dark Eden Origin,SOFTON ENTERTAINMENT,SOFTON ENTERTAINMENT,NaN,36,51,0,"20,000 .. 50,000",77,0,77,0,0,0,0,0,20000,50000,35000.0
3960,523190,Weapons of Mythology - New Age -,Gamemag,IDC/Games,NaN,30,46,0,"20,000 .. 50,000",193,0,362,0,0,0,0,0,20000,50000,35000.0


In [ ]:
import pandas as pd
import requests
import time
import random
from tqdm import tqdm
import os
INPUT_FILE = "steamspy_cleaned_final.csv"
OUTPUT_FILE = "steamspy_enriched.csv"
TEMP_FILE = "checkpoint_partial.csv"
PAUSE_MIN = 0.8
PAUSE_MAX = 1.5
SAVE_EVERY = 200   # guarda cada X juegos

#  Cargar dataset base 
df = pd.read_csv(INPUT_FILE)
df["appid"] = df["appid"].astype(str)

#  Reanudar si hay checkpoint previo
if os.path.exists(TEMP_FILE):
    df_api = pd.read_csv(TEMP_FILE)
    processed_ids = set(df_api["appid"])
    print(f"🔁 Reanudando desde checkpoint ({len(processed_ids)} juegos ya completados)")
else:
    df_api = pd.DataFrame(columns=["appid", "release_date", "genres", "is_free", "metacritic_score", "dlc_count"])
    processed_ids = set()

#  Obtener detalles desde Steam API
def get_steam_details(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json().get(appid, {}).get("data", {})
        if not data:
            return None

        release_date = data.get("release_date", {}).get("date", None)
        genres = ", ".join([g["description"] for g in data.get("genres", [])]) if data.get("genres") else None
        is_free = data.get("is_free", None)
        metacritic = data.get("metacritic", {}).get("score", None)
        dlc_count = len(data.get("dlc", [])) if data.get("dlc") else 0

        return {
            "appid": appid,
            "release_date": release_date,
            "genres": genres,
            "is_free": is_free,
            "metacritic_score": metacritic,
            "dlc_count": dlc_count
        }
    except Exception as e:
        print(f" Error con appid {appid}: {e}")
        return None

# --- Recorrer appids con barra de progreso ---
for i, appid in enumerate(tqdm(df["appid"], desc="Enriqueciendo juegos", unit="game")):
    if appid in processed_ids:
        continue

    info = get_steam_details(appid)
    if info:
        df_api = pd.concat([df_api, pd.DataFrame([info])], ignore_index=True)

    # Pausa aleatoria segura
    time.sleep(random.uniform(PAUSE_MIN, PAUSE_MAX))

    # Guardar checkpoint cada cierto número
    if (len(df_api) % SAVE_EVERY == 0) and (len(df_api) > 0):
        df_api.to_csv(TEMP_FILE, index=False)
        print(f" Guardado parcial ({len(df_api)} juegos)")

# --- Guardado final ---
df_api.to_csv(TEMP_FILE, index=False)
df_enriched = df.merge(df_api, on="appid", how="left")
df_enriched.to_csv(OUTPUT_FILE, index=False)
os.remove(TEMP_FILE)

print(f"\n Enriquecimiento completado y guardado en '{OUTPUT_FILE}'")


Enriqueciendo juegos:   0%|          | 0/2446 [00:01<?, ?game/s]


KeyboardInterrupt: 

In [7]:
df_enriched=pd.read_csv("steamspy_enriched.csv")
df_enriched

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,...,discount,ccu,owners_min,owners_max,owners_median,release_date,genres,is_free,metacritic_score,dlc_count
0,1172470,Apex Legends,Respawn,Electronic Arts,NaN,668053,326926,0,"100,000,000 .. 200,000,000",11025,...,0,124262,100000000,200000000,150000000.0,"4 Nov, 2020","Action, Adventure, Free To Play",True,88.0,0.0
1,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",NaN,1520457,1037487,0,"100,000,000 .. 200,000,000",23333,...,0,314682,100000000,200000000,150000000.0,"21 Dec, 2017","Action, Adventure, Massively Multiplayer, Free...",True,NaN,0.0
2,440,Team Fortress 2,Valve,Valve,NaN,1044264,117208,0,"50,000,000 .. 100,000,000",21058,...,0,43819,50000000,100000000,75000000.0,"10 Oct, 2007","Action, Free To Play",True,92.0,1.0
3,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,NaN,506516,48852,0,"50,000,000 .. 100,000,000",9607,...,0,10408,50000000,100000000,75000000.0,"7 Jul, 2017","Action, Adventure, Casual, Indie, Free To Play",True,NaN,1.0
4,1599340,Lost Ark,Smilegate RPG,Amazon Games,NaN,143481,58540,0,"50,000,000 .. 100,000,000",5408,...,0,17355,50000000,100000000,75000000.0,"11 Feb, 2022","Action, Adventure, Massively Multiplayer, RPG,...",True,NaN,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441,796380,Terra Feminarum,Polar Night Games,Polar Night Games,NaN,54,8,0,"20,000 .. 50,000",278,...,0,0,20000,50000,35000.0,"28 Feb, 2018","Action, Indie",True,NaN,1.0
2442,651280,TinkerQuarry,Hallowraith,Caprizant Games,NaN,81,8,0,"20,000 .. 50,000",120,...,0,0,20000,50000,35000.0,"13 Sep, 2019","Adventure, Indie, RPG",False,NaN,0.0
2443,658510,Dark Eden Origin,SOFTON ENTERTAINMENT,SOFTON ENTERTAINMENT,NaN,36,51,0,"20,000 .. 50,000",77,...,0,0,20000,50000,35000.0,"17 Aug, 2017","Free To Play, Massively Multiplayer, RPG",True,NaN,0.0
2444,523190,Weapons of Mythology - New Age -,Gamemag,IDC/Games,NaN,30,46,0,"20,000 .. 50,000",193,...,0,0,20000,50000,35000.0,"15 Dec, 2016","Adventure, Free To Play, Massively Multiplayer...",False,NaN,0.0


In [8]:
df_enriched["release_date"].head(10)


0     4 Nov, 2020
1    21 Dec, 2017
2    10 Oct, 2007
3     7 Jul, 2017
4    11 Feb, 2022
5    15 Aug, 2013
6    25 Mar, 2013
7     6 Dec, 2024
8    16 Nov, 2018
9    17 Oct, 2017
Name: release_date, dtype: object

In [9]:
import pandas as pd


# --- 1️⃣ Quitar duplicados y normalizar columnas ---
df_enriched = df_enriched.drop_duplicates(subset="appid")
df_enriched["release_date"] = df_enriched["release_date"].astype(str).str.strip()

# --- 2️⃣ Convertir fecha y extraer el año ---
# Formato: "4 Nov, 2020"
df_enriched["release_date"] = pd.to_datetime(df_enriched["release_date"], format="%d %b, %Y", errors="coerce")
df_enriched["release_year"] = df_enriched["release_date"].dt.year

# --- 3️⃣ Limpiar y tipar columnas clave ---
df_enriched["is_free"] = df_enriched["is_free"].fillna(False).astype(bool)
df_enriched["dlc_count"] = pd.to_numeric(df_enriched["dlc_count"], errors="coerce").fillna(0).astype(int)
df_enriched["metacritic_score"] = pd.to_numeric(df_enriched["metacritic_score"], errors="coerce")

# --- 4️⃣ Eliminar juegos sin año o con fechas fuera del rango ---
df_enriched = df_enriched.dropna(subset=["release_year"])
df_enriched = df_enriched[df_enriched["release_year"] >= 2000]  # centrarse en el modelo moderno

# --- 5️⃣ Clasificar el modelo de negocio ---
df_enriched["business_model"] = df_enriched.apply(
    lambda row: (
        "Free-to-play" if row["is_free"]
        else "Hybrid (Paid + DLC)" if row["dlc_count"] > 0
        else "Paid"
    ),
    axis=1
)

C:\Users\migue\AppData\Local\Temp\ipykernel_26012\2453889864.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_enriched["is_free"] = df_enriched["is_free"].fillna(False).astype(bool)


NameError: name 'df_total' is not defined

In [14]:
df_enriched.columns

Index(['appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
       'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'owners_min', 'owners_max', 'owners_median', 'release_date',
       'genres', 'is_free', 'metacritic_score', 'dlc_count', 'release_year',
       'business_model'],
      dtype='object')

In [15]:


# 1. Ver los nombres de todas las columnas
print(df_enriched.columns)

# 2. Ver como lista
print(df_enriched.columns.tolist())

# 3. Configurar pandas para mostrar todas las columnas al imprimir
pd.set_option('display.max_columns', None)
print(df_enriched)

# 4. Ver información detallada de todas las columnas
print(df_enriched.info())

# 5. Ver solo los nombres en formato vertical (más legible)
for col in df_enriched.columns:
    print(col)

# 6. Temporalmente mostrar todas las columnas (sin cambiar configuración global)
with pd.option_context('display.max_columns', None):
    print(df_enriched.head())

Index(['appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
       'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'owners_min', 'owners_max', 'owners_median', 'release_date',
       'genres', 'is_free', 'metacritic_score', 'dlc_count', 'release_year',
       'business_model'],
      dtype='object')
['appid', 'name', 'developer', 'publisher', 'score_rank', 'positive', 'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount', 'ccu', 'owners_min', 'owners_max', 'owners_median', 'release_date', 'genres', 'is_free', 'metacritic_score', 'dlc_count', 'release_year', 'business_model']
        appid                              name              developer  \
0     1172470                      Apex Legends                Respawn   
1      578080               PUBG: BATTLEGROUNDS 